# Text Pre Processor - Implantação

Este é um componente que utiliza a biblioteca [nltk](https://www.nltk.org/) e [ftfy](https://pypi.org/project/ftfy/) e [regex](https://docs.python.org/3/library/re.html) para pré processar textos que entrrão em outros componentes.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import pandas as pd
import numpy as np
from pre_processor import Preprocessor

class Model:
    
    def __init__(self):
        self.loaded = False
        
        
    def load(self):

        artifacts = joblib.load("/tmp/data/preprocessor.joblib")
        self.preprocessing_tasks = artifacts["preprocessing_tasks"]
        self.model_parameters = artifacts["model_parameters"]
        self.columns = artifacts["columns"]
        self.target = artifacts["target"]

        self.preprocessor = Preprocessor(self.preprocessing_tasks, self.model_parameters)

        self.loaded = True
            
    def class_names(self):
        return ['text']

    
    def predict(self, X, feature_names, meta=None):

        if not self.loaded:
            self.load()
        
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=self.columns)
        
        else:
            if isinstance(X, str):
                df = pd.DataFrame({self.target: [X]})
            else:
                df = pd.DataFrame({self.target: X})
            
        texts = self.preprocessor.preprocess(df[self.target].to_numpy())
        texts = np.array(texts)

        if len(texts.shape) == 1:
            texts = np.expand_dims(texts, axis=0) 
        
        return texts